# VRP GraphNet

model inputs from the paper:

| Variable             | Meaning                           | Dimensions                |
|----------------------|-----------------------------------|---------------------------|
| batch_edges          | Adj matrix special connections*   | B x num_nodes x num_nodes |
| batch_edges_values   | Distance Matrix                   | B x num_nodes x num_nodes |
| batch_edges_target   | Target adj matrix                 | B x num_nodes x num_nodes |
| batch_nodes          | Ones vector                       | B x num_nodes             |
| batch_nodes_coord    | Coordinates                       | B x num_nodes x 2         |
| *batch_nodes_target* | Value represents ordering in tour | B x num_nodes             |


*special connections:
* 1 - k-nearest neighbour
* 2 - self connections
* 0 - otherwise

In [ ]:
try:
    # noinspection PyUnresolvedReferences
    from google.colab import drive

    drive.mount("/content/gdrive")

    %cd gdrive/My Drive/vrp-thesis
    %pip install -r requirements-colab.txt
    IN_COLAB = True
except:
    IN_COLAB = False

In [ ]:
if IN_COLAB:
    %reload_ext tensorboard
    %tensorboard --logdir runs

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from model import GraphNet
from utils import (
    load_config,
    get_metrics,
    get_device,
    save_checkpoint,
    DotDict,
    load_checkpoint,
    BeamSearch,
)

# noinspection PyUnresolvedReferences
from utils.data import (
    load_and_split_dataset,
    process_datasets,
    adj_matrix_from_routes,
    distance_from_adj_matrix,
)

sns.set_theme()

## Load datasets

In [ ]:
dsets = load_and_split_dataset("data/vrp_20_3s_random_depot.pkl", test_size=500)
train_dataset, test_dataset = process_datasets(dsets, k=6)

print(len(train_dataset), len(test_dataset))

## Basic Config

In [ ]:
device = get_device()
print("Device", device)

In [ ]:
config = load_config(
    hidden_dim=32,
    num_gcn_layers=5,
    num_mlp_layers=3,
    learning_rate=0.001,
    train_batch_size=64,
    test_batch_size=256,
    num_epochs=50,
)
config

In [ ]:
train_dataloader = DataLoader(
    train_dataset, batch_size=config.train_batch_size, shuffle=True
)

model = GraphNet(config).to(device)

## Test Forward Pass

In [ ]:
features, _ = next(iter(train_dataloader))

y_pred = model.forward(
    features["node_features"].to(device),
    features["dist_matrix"].to(device),
    features["edge_feat_matrix"].to(device),
)

y_pred.shape

## Validation loop

In [ ]:
def exceeds_capacity(tour, demand):
    loads = np.take(demand, tour)

    running_load = 0

    for j in range(len(loads)):
        running_load += loads[j]

        if tour[j] == 0 or j == len(loads) - 1:
            if np.round(running_load, 2) > 1.0:
                return True
            running_load = 0

    return False


def is_valid(tours, demands, num_nodes):
    assert (
        tours.shape[0] == demands.shape[0]
    ), "Batch size of tours and demands must match"
    assert isinstance(tours, np.ndarray) and isinstance(
        demands, np.ndarray
    ), "tours and demands must be numpy arrays"

    valid = np.ones(tours.shape[0], dtype=bool)

    for i, tour in enumerate(tours):
        node_visit_count = np.bincount(tour, minlength=num_nodes)
        _valid_capacity = not exceeds_capacity(tour, demands[i])
        _valid_tour = np.all(node_visit_count[1:] == 1)

        valid[i] = _valid_capacity and _valid_tour

    return valid


def shortest_valid_tour(
    y_preds,
    batch_dist_matrix,
    batch_node_features,
    num_vehicles=None,
    beam_width=1024,
    allow_consecutive_visits=False,
):
    # Move tensors to CPU for faster computation (due to loops and compare ops)
    y_preds = y_preds.cpu()
    batch_demands = batch_node_features[..., 2].cpu()

    y_preds = y_preds[..., 1]

    beamsearch = BeamSearch(
        y_preds,
        demands=batch_demands if num_vehicles is None else None,
        num_vehicles=num_vehicles or 0,
        allow_consecutive_visits=allow_consecutive_visits,
        beam_width=beam_width,
    )
    beamsearch.search()

    batch_dist_matrix = batch_dist_matrix.cpu().numpy()
    batch_demands = batch_demands.numpy()

    shortest_tour = np.zeros(
        (beamsearch.batch_size, len(beamsearch.next_nodes)), dtype=int
    )
    shortest_tour_length = np.full((beamsearch.batch_size,), np.inf)

    for b in range(beamsearch.beam_width):
        current_tour = beamsearch.get_beam(b)
        current_tour = current_tour.numpy()

        adj_matrix = adj_matrix_from_routes(current_tour, beamsearch.num_nodes)
        tour_length = distance_from_adj_matrix(adj_matrix, batch_dist_matrix)
        valid = is_valid(current_tour, batch_demands, beamsearch.num_nodes)

        for i in range(beamsearch.batch_size):
            if valid[i] and tour_length[i] < shortest_tour_length[i]:
                shortest_tour[i] = current_tour[i]
                shortest_tour_length[i] = tour_length[i]

    return shortest_tour, shortest_tour_length


def greedy_tour_lengths(y_preds, batch_dist_matrix, batch_node_features):
    y_preds = y_preds.cpu()
    batch_demands = batch_node_features[..., 2].cpu()

    # only keep the probability of selecting the edge
    y_preds = y_preds[..., 1]

    beamsearch = BeamSearch(
        y_preds, demands=batch_demands, beam_width=1, num_vehicles=0
    )
    beamsearch.search()

    # get most probable tours (index = 0)
    tours = beamsearch.get_beam(0)

    tours = tours.cpu().numpy()
    batch_dist_matrix = batch_dist_matrix.cpu().numpy()

    __adj_matrix = adj_matrix_from_routes(tours, batch_dist_matrix.shape[-1])
    tour_lengths = distance_from_adj_matrix(__adj_matrix, batch_dist_matrix)

    return tours, tour_lengths


def eval_model(batch_node_features, batch_dist_matrix, batch_edge_features, model):
    model.eval()

    with torch.no_grad():
        preds = model(batch_node_features, batch_dist_matrix, batch_edge_features)
        preds = F.softmax(preds, dim=3)

        return preds


def validate(dataloader, model, criterion):
    running_loss = 0
    running_tour_lengths = 0
    running_opt_gap = 0
    running_tour_count = 0
    targets = []
    predictions = []

    for batch_features, batch_targets in dataloader:
        batch_node_features = batch_features["node_features"].to(device)
        batch_dist_matrix = batch_features["dist_matrix"].to(device)
        batch_edge_features = batch_features["edge_feat_matrix"].to(device)
        batch_targets = batch_targets.to(device)

        y_preds = eval_model(
            batch_node_features, batch_dist_matrix, batch_edge_features, model=model
        )

        # Loss
        loss = get_loss(y_preds, batch_targets, criterion)
        running_loss += loss.item()

        opt_tour_lengths = distance_from_adj_matrix(batch_targets, batch_dist_matrix)
        # Greedy (cap) tour lengths
        _, tour_lengths = greedy_tour_lengths(
            y_preds, batch_dist_matrix, batch_node_features
        )

        opt_gap = (tour_lengths / opt_tour_lengths) - 1

        running_tour_lengths += tour_lengths.sum()
        running_tour_count += len(tour_lengths)
        running_opt_gap += opt_gap.sum()

        y_preds = y_preds.argmax(dim=3)
        y_preds = y_preds.cpu().numpy()

        targets.append(batch_targets.cpu().numpy())
        predictions.append(y_preds)

    targets = np.concatenate(targets)
    predictions = np.concatenate(predictions)
    mean_running_loss = running_loss / len(dataloader)
    mean_tour_lengths = running_tour_lengths / running_tour_count
    mean_opt_gap = running_opt_gap / running_tour_count

    return (targets, predictions, mean_running_loss, mean_tour_lengths, mean_opt_gap)

## Training Loop

In [ ]:
def get_loss(preds, targets, criterion):
    preds_perm = preds.permute(0, 3, 1, 2)

    return criterion(preds_perm, targets)


def train_one_epoch(dataloader, model, optimizer, criterion):
    running_loss = 0

    model.train()

    for batch_idx, (batch_features, batch_targets) in enumerate(dataloader):
        optimizer.zero_grad()

        batch_node_features = batch_features["node_features"].to(device)
        batch_dist_matrix = batch_features["dist_matrix"].to(device)
        batch_edge_features = batch_features["edge_feat_matrix"].to(device)
        batch_targets = batch_targets.to(device)

        preds = model(batch_node_features, batch_dist_matrix, batch_edge_features)
        loss = get_loss(preds, batch_targets, criterion)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()

    return running_loss


def train(num_epochs, train_dl, test_dl, model, optimizer, criterion, writer):
    best_loss = np.inf

    for epoch in range(num_epochs):
        # Train
        running_loss = train_one_epoch(
            train_dl, model=model, optimizer=optimizer, criterion=criterion
        )

        # Losses
        epoch_loss = running_loss / len(train_dl)

        # Validation Metrics
        (
            targets,
            predictions,
            validation_loss,
            mean_tour_length,
            mean_opt_gap,
        ) = validate(test_dl, model=model, criterion=criterion)
        metrics = get_metrics(targets, predictions)

        writer.add_scalar("Metrics/accuracy", metrics.acc, epoch)
        writer.add_scalar("Metrics/bal. accuracy", metrics.bal_acc, epoch)
        writer.add_scalar("Metrics/precision", metrics.precision, epoch)
        writer.add_scalar("Metrics/recall", metrics.recall, epoch)
        writer.add_scalar("Metrics/f1 score", metrics.f1_score, epoch)
        writer.add_scalar("Metrics/mean tour length", mean_tour_length, epoch)
        writer.add_scalar("Metrics/mean opt. gap", mean_opt_gap, epoch)

        writer.add_scalar("Loss/train", epoch_loss, epoch)
        writer.add_scalar("Loss/test", validation_loss, epoch)

        # Save (validation) checkpoint
        if validation_loss < best_loss:
            best_loss = validation_loss
            save_checkpoint(
                writer.log_dir / "best_validation_loss_model.pt",
                model=model,
                optimizer=optimizer,
                epoch=epoch,
                config={**config},
                train_loss=epoch_loss,
                test_loss=validation_loss,
            )

        # Save (epoch) checkpoint
        save_checkpoint(
            writer.log_dir / "last_epoch_model.pt",
            model=model,
            optimizer=optimizer,
            epoch=epoch,
            config={**config},
            train_loss=epoch_loss,
            test_loss=validation_loss,
        )

        print(f"Epoch: {epoch:02d}, Loss: {epoch_loss:.4f}")

## Baseline Model

In [ ]:
LOG_DIR = Path(f"runs/exp_baseline_1")

config = load_config(
    hidden_dim=16,
    num_gcn_layers=5,
    num_mlp_layers=3,
    learning_rate=0.001,
    train_batch_size=64,
    test_batch_size=256,
    num_epochs=10,
)

train_dataloader = DataLoader(
    train_dataset, batch_size=config.train_batch_size, shuffle=True
)
test_dataloader = DataLoader(
    test_dataset, batch_size=config.test_batch_size, shuffle=True
)

torch.manual_seed(0)

edge_class_weights = train_dataset.class_weights().to(device)
model = GraphNet(config).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
criterion = nn.CrossEntropyLoss(edge_class_weights)

writer = SummaryWriter(log_dir=LOG_DIR)

train(
    config.num_epochs,
    train_dl=train_dataloader,
    test_dl=test_dataloader,
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
)
writer.flush()
writer.close()

## Plot Results

In [ ]:
MODEL_PATH = Path("runs/exp_baseline_1")

checkpoint = load_checkpoint(MODEL_PATH / "last_epoch_model.pt")
config = DotDict(checkpoint["config"])
model = GraphNet(config).to(device)

model.load_state_dict(checkpoint["model_state_dict"])

In [ ]:
test_dataloader = DataLoader(
    test_dataset, batch_size=config.test_batch_size, shuffle=True
)
batch_features, batch_targets = next(iter(test_dataloader))

In [ ]:
batch_node_features = batch_features["node_features"].to(device)
batch_dist_matrix = batch_features["dist_matrix"].to(device)
batch_edge_features = batch_features["edge_feat_matrix"].to(device)
batch_targets = batch_targets.to(device)

preds = eval_model(
    batch_node_features, batch_dist_matrix, batch_edge_features, model=model
)

In [ ]:
routes, distances = shortest_valid_tour(
    preds,
    batch_dist_matrix,
    batch_node_features,
    num_vehicles=None,
    beam_width=1024,
    allow_consecutive_visits=False,
)

In [ ]:
1 - np.isinf(distances).sum() / len(distances)

In [ ]:
for c in np.random.choice(len(batch_node_features), 5):
    fig, ax = plt.subplots(1, 2, figsize=(15, 5))
    sns.heatmap(batch_targets[c].cpu().numpy(), ax=ax[0])
    sns.heatmap(preds[c, ..., 1].cpu().numpy(), ax=ax[1])

    plt.show()

In [ ]:
from utils.plot import plot_graph, plot_heatmap, plot_beam_search_tour

NUM_SAMPLES = 10

actual_distance = distance_from_adj_matrix(batch_targets, batch_dist_matrix)
# routes, distances = greedy_tour_lengths(preds, batch_dist_matrix, batch_node_features)
routes, distances = shortest_valid_tour(
    preds, batch_dist_matrix, batch_node_features, None
)

actual_distance = actual_distance.cpu().numpy()
node_features = batch_node_features.cpu().numpy()
targets = batch_targets.cpu().numpy()
predictions = preds[..., 1].cpu().numpy()

for i in np.random.choice(len(node_features), NUM_SAMPLES):
    print(f"Sample {i}")
    fig, ax = plt.subplots(1, 3, figsize=(15, 5))

    plot_graph(node_features[i, :, :2], targets[i], ax=ax[0])
    plot_heatmap(node_features[i, :, :2], targets[i], predictions[i], ax=ax[1])
    plot_beam_search_tour(node_features[i, :, :2], targets[i], routes[i], ax=ax[2])

    ax[0].set_title(f"Ground truth ({actual_distance[i]:.2f})")
    ax[1].set_title("Predictions")
    ax[2].set_title(f"Shortest tour ({distances[i]:.2f})")
    fig.tight_layout()

    plt.show()